In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

- [PyTorch에서 추론을 위해 모델 저장하기 & 불러오기](https://tutorials.pytorch.kr/recipes/recipes/saving_and_loading_models_for_inference.html)
- PyTorch에서는 추론(inference)을 위해 모델을 저장하고 불러오는데 2가지 접근법이 있음
    - 1) 첫 번째는 `state_dict`를 저장하고 불러오는 것
    - 2) 두 번째는 전체 모델을 저장하는 것

# 개요
- torch.save() 함수를 사용하여 모델의 `state_dict`를 저장하면 이후에 모델을 불러올 때 유연함을 크게 살릴 수 있음
- 학습된 모델의 매개변수(parameter)만을 저장하면 되므로 모델 저장 시에 권장되는 방법임
- 모델 전체를 저장하고 불러올 때에는 Python의 pickle 모듈을 사용하여 전체 모듈을 저장함
    - 이 방식은 직관적인 문법을 사용하며 코드의 양도 적음
    - 단점은 직렬화(serialized)된 데이터가 모델을 저장할 때 사용한 특정 클래스 및 디렉토리 구조에 종속(bind)된다는 것임
        - 그 이유는 pickle이 모델 클래스 자체를 저장하지 않고 불러올 때 사용되는 클래스가 포함된 파일의 경로를 저장함 
        - 이 때문에 작성한 코드가 다른 프로젝트에서 사용되거나 리팩토링을 거치는 등의 과정에서 동작하지 않을 수 있음
- 추론을 위해 모델을 저장하고 불러오는 두 가지 방법 모두를 살펴볼 것임

# 단계(Steps)
1. 데이터 불러올 때 필요한 라이브러리들 불러오기
2. 신경망을 구성하고 초기화하기
3. 옵티마이저 초기화하기
4. `state_dict`를 통해 모델을 저장하고 불러오기
5. 전체 모델을 저장하고 불러오기

## 데이터 불러올 때 필요한 라이브러리들 불러오기
- `torch`와 여기 포함된 `torch.nn`과 `torch.optim`을 사용할 것임

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

## 신경망을 구성하고 초기화하기
- 예를 들기 위해, 이미지를 학습하는 신경망을 만들어볼 것임

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pook(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## 옵티마이저 초기화하기
- momentum을 갖는 SGD를 사용할 것임

In [6]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [19]:
# net.parameters() : 매 층의 parameters
params = list(net.parameters())
for i in range(len(params)):
    params[i].shape

torch.Size([6, 3, 5, 5])

torch.Size([6])

torch.Size([16, 6, 5, 5])

torch.Size([16])

torch.Size([120, 400])

torch.Size([120])

torch.Size([84, 120])

torch.Size([84])

torch.Size([10, 84])

torch.Size([10])

## `state_dict`를 통해 모델을 저장하고 불러오기

In [22]:
# 경로 지정
PATH = 'state_dict_model.pt'

# 저장하기
torch.save(net.state_dict(), PATH)

# 불러오기
model = Net()
model.load_state_dict(torch.load(PATH))
model.eval()

<All keys matched successfully>

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

- PyTorch에는 `.pt` 또는 `.pth` 확장자를 사용하는 것이 일반적인 규칙임
- `load_state_dict()` 함수는 저장된 객체의 경로가 아닌 사전 객체를 사용함
    - 즉, 저장된 state_dict를 `load_state_dict()` 함수에 전달하기 전에 반드시 역직렬화(deserialize)를 해야 함
    - 예를 들어, `model.load_state_dict(PATH)`와 같이 사용할 수 없음
- 추론을 수행하기 전에 `model.eval()`을 호출하여 dropout과 batch normalization layers를 evaluation 모드로 바꿔야 함을 기억하기!
    - 이를 빼먹으면 일관성 없는 추론 결과를 얻게 됨

## 전체 모델을 저장하고 불러오기

In [23]:
# 경로 지정
PATH = 'entire_model.pt'

# 저장하기
torch.save(net, PATH)

# 불러오기
model = torch.load(PATH)
model.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

여기서도 또한 `model.eval()`을 실행하여 dropout과 batch normalization layers를 evaluation 모드로 바꿔야한다는 것을 기억하기